In [ ]:
import boto3
import configparser

#setup_configuration
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY = config.get('default','aws_access_key_id')
SECRETE = config.get('default','aws_secret_access_key')

In [ ]:
#creating redshift client
redshift = boto3.client('redshift',region_name = 'us-east-1',aws_access_key_id=KEY,aws_secret_access_key=SECRETE)

In [ ]:
def create_cluster(redshift, DWH_CLUSTER_TYPE, NUM_NODES, DB_NAME, DB_USER, DB_PASSWORD):
    cluster_type = 'multi-node' if NUM_NODES > 1 else 'single-node'
    response = redshift.create_cluster(
        ClusterType=cluster_type,
        NodeType=DWH_CLUSTER_TYPE,
        NumberOfNodes=NUM_NODES,
        DBName=DB_NAME,
        ClusterIdentifier='my-redshift-cluster',
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        AutomatedSnapshotRetentionPeriod=2,
        VpcSecurityGroupIds=['sg-0f3249e4995f4133b'],
        ClusterSubnetGroupName='default'
    )
    print('Cluster ARN:', response['Cluster']['ClusterArn'])

create_cluster(redshift, 'dc2.large', 2, 'development', 'awsuser', 'Prakash-123456')


In [ ]:
# def create_security_group(redshift,security_group_name,vpc_id,port):
#     response= redshift.authorize_cluster_security_group_ingress(
#         ClusterIdentifier = 'my-redshift-cluster',
#         ClusterSecurityGroupName = security_group_name,
#         CIDRIP = '0.0.0.0/0',
#         FromPort =port,
#         ToPort = port
#     )
#     print('Security group ID:', response['ClusterSecurityGroupName'])
# create_security_group(redshift, 'model-check','vpc-0b1c21df24b40d857',5439)



In [ ]:
import boto3

def create_security_group(ec2, security_group_id, port):
    response = ec2.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=[
            {
                'IpProtocol': 'tcp',
                'FromPort': port,
                'ToPort': port,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
            }
        ]
    )
    print('Ingress rule added to security group:', security_group_id)

# Specify the region when creating the EC2 client
ec2 = boto3.client('ec2', region_name='us-east-1')

# Replace 'your-security-group-id' with your actual security group ID and '5439' with your desired port number
create_security_group(ec2, 'sg-0b7be7c1918181e69', 5439)


In [ ]:

try:
    import redshift_connector
except ModuleNotFoundError:
    !pip install redshift_connector
    import redshift_connector

In [6]:
conn = redshift_connector.connect(
    host = 'my-redshift-cluster.cst1nhbptfv8.us-east-1.redshift.amazonaws.com',
    database = 'development',
    port = 5439,
    user = 'awsuser',
    password = 'Prakash-123456'
)
cursor  = conn.cursor()
print('success')

success


In [15]:
conn.rollback()

In [7]:
cursor.execute("ROLLBACK;")

In [8]:
cursor.execute("""
CREATE TABLE employement_data (
    id INT IDENTITY(1,1) PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    department VARCHAR(50),
    salary DECIMAL(10, 2),
    hire_date DATE,
    job_title VARCHAR(50),
    location VARCHAR(50),
    phone_number VARCHAR(20)
)
""")
conn.commit()


# result:tuple = cursor.fetchall()
# print(result)

In [9]:
cursor.execute("""
INSERT INTO employement_data (first_name, last_name, email, department, salary, hire_date, job_title, location, phone_number)
VALUES 
    ('John', 'Doe', 'john.doe@example.com', 'Engineering', 75000.00, '2023-01-01', 'Software Engineer', 'New York', '123-456-7890'),
    ('Jane', 'Smith', 'jane.smith@example.com', 'Marketing', 65000.00, '2023-02-01', 'Marketing Manager', 'San Francisco', '987-654-3210'),
    ('Bob', 'Johnson', 'bob.johnson@example.com', 'Sales', 55000.00, '2023-03-01', 'Sales Associate', 'Los Angeles', '456-789-0123')
""")
conn.commit()

In [17]:
cursor.execute("""Select * from employement_data""")
result:tuple = cursor.fetchall()
print(result)
# conn.commit()

([1, 'John', 'Doe', 'john.doe@example.com', 'Engineering', Decimal('75000.00'), datetime.date(2023, 1, 1), 'Software Engineer', 'New York', '123-456-7890'], [2, 'Jane', 'Smith', 'jane.smith@example.com', 'Marketing', Decimal('65000.00'), datetime.date(2023, 2, 1), 'Marketing Manager', 'San Francisco', '987-654-3210'], [3, 'Bob', 'Johnson', 'bob.johnson@example.com', 'Sales', Decimal('55000.00'), datetime.date(2023, 3, 1), 'Sales Associate', 'Los Angeles', '456-789-0123'])
